In [1]:
import pandas as pd
import numpy as np
import re

In [71]:
zipcode = input('input zipcode')
df = pd.read_csv('%s/%sdata.csv' % (zipcode, zipcode))
print('before', len(df))
df = df.drop_duplicates()

# 刪除重複的標題
index = df[df['mls'] == 'mls'].index.to_list()
df.drop(labels=index, axis='index', inplace=True)

# 刪除houseprice為空值
index = df[df['houseprice'].isnull() == True].index.to_list()
df.drop(labels=index, axis='index',inplace=True)

df.reset_index(inplace=True, drop=True)
print('after: ',len(df))

# lot acres to sqft
for i in range(len(df)):
    if re.search('Acres', str(df.iloc[i]['lot'])) != None:
        lot = float(re.search('[0-9]+\.?[0-9]*',df.iloc[i]['lot']).group())
        lot = int(lot * 43560)
        df.iloc[i, 13] = lot
    if re.search('sqft', str(df.iloc[i]['lot'])) != None:
        lot = int(re.search('[0-9]+,?[0-9]*',df.iloc[i]['lot']).group().replace(',',''))
        df.iloc[i, 13] = lot
# hoa
for i in range(len(df)):
    hoa = str(df.iloc[i]['hoa']).replace(',', '')
    df.iloc[i,16] = hoa

        
# No Data to Null
index = df[df['houseyear'] == 'No Data'] .index.to_list()
for i in index:
    df.iloc[i, 12] = None
    
print(df.isnull().sum())


input zipcode 98101


before 8
after:  8
mls              0
housebedroom     0
housebathroom    1
sqft             1
address          0
soldout          0
walkscore        0
transitscore     0
school1          2
school2          2
school3          3
housetype        0
houseyear        3
lot              5
pt               3
ta               3
hoa              0
houseprice       0
dtype: int64


In [72]:
# 換成float
df[['housebedroom','housebathroom', 'sqft', 'walkscore', 'transitscore', 'school1', 'school2', 'school3', 'lot', 'pt','ta', 'hoa', 'houseprice']] = df[['housebedroom','housebathroom', 'sqft', 'walkscore', 'transitscore', 'school1', 'school2', 'school3', 'lot', 'pt','ta', 'hoa', 'houseprice']].astype('float')

# 空值補0
zeros = ['housebedroom', 'housebathroom', 'walkscore', 'transitscore', 'school1', 'school2', 'school3', 'lot']
for zero in zeros:
    df[zero].fillna(0,inplace=True)
    
# 空值補平均
df['sqft'].fillna(value=df['sqft'].mean(), inplace=True)

# 空值補中位數
df['houseyear'].fillna(value=df['houseyear'].median(), inplace=True)


# 按照計算公式補值
ptIndex = df[df['pt'].isnull() == True].index.to_list()
for index in ptIndex:
    ptFillna = df['pt'].mean() / df['houseprice'].mean() *df.iloc[index]['houseprice']
    df.iloc[index, 14] = int(ptFillna)
    
taIndex = df[df['ta'].isnull() == True].index.to_list()
for index in taIndex:
    taFillna = df['ta'].mean() / df['houseprice'].mean() *df.iloc[index]['houseprice']
    df.iloc[index, 15] = int(taFillna)
    
print(df.isnull().sum())

#to int
df[['sqft', 'walkscore', 'transitscore', 'school1', 'school2', 'school3', 'lot', 'pt','ta', 'houseprice']] = df[['sqft', 'walkscore', 'transitscore', 'school1', 'school2', 'school3', 'lot', 'pt','ta', 'houseprice']].astype('int')




mls              0
housebedroom     0
housebathroom    0
sqft             0
address          0
soldout          0
walkscore        0
transitscore     0
school1          0
school2          0
school3          0
housetype        0
houseyear        0
lot              0
pt               0
ta               0
hoa              6
houseprice       0
dtype: int64


In [73]:
df.to_csv('%s/%sdataclean.csv' % (zipcode, zipcode), index=0)